# Advent of Code 2023
## Day 19
*<https://adventofcode.com/2023/day/19>*

In [1]:
import heapq
import math
import re
import functools as ft
from collections import Counter, defaultdict, deque, namedtuple
from itertools import combinations, permutations, product
from string import ascii_letters, ascii_lowercase, ascii_uppercase

import IPython
import z3
from rich import inspect, pretty, print

from new_helper import *

pretty.install()

In [2]:
DAY = 19
input_str = get_aoc_input(DAY, 2023)
part_1 = part_2 = 0

In [3]:
inp = input_str.parse_groups()

In [4]:
class Guard:
    var: str
    gt: bool
    val: int

    def __init__(self, var: str, gt: bool, val: int):
        self.var = var
        self.gt = gt
        self.val = val
    
    def __repr__(self):
        return f"{self.var} {'>' if self.gt else '<'} {self.val}"

    def inverse(self):
        return Guard(self.var, not self.gt, self.val + 1 if self.gt else self.val - 1)

In [5]:
in_workflows, in_ratings = inp
workflows: dict[str, list[tuple[Guard | None, str]]] = {}

for w in in_workflows:
    name, rules = w.split("{")
    rules = rules[:-1]
    rs = []
    for r in rules.split(","):
        if ":" in r:
            gt = ">" in r
            s, res = r.split(":")
            workflow_name, val = s.split(">" if gt else "<")
            val = int(val)
            rs.append((Guard(workflow_name, gt, val), res))
        else:
            rs.append((None, r))

    workflows[name] = rs

In [6]:
def process(work: str, vars: dict[str, int]):
    if work in "AR":
        return work

    workflow = workflows[work]
    for g, res in workflow:
        if g is None:
            return process(res, vars)

        if g.gt and vars[g.var] > g.val:
            return process(res, vars)
        elif not g.gt and vars[g.var] < g.val:
            return process(res, vars)
    
    assert False

In [7]:
for r in in_ratings:
    r = r[1:-1]
    d = {s.split("=")[0]: int(s.split("=")[1]) for s in r.split(",")}

    if process("in", d) == "A":
        part_1 += sum(d.values())

In [8]:
MIN = 1
MAX = 4000


def ranges_for_name(workflow_name: str) -> list[dict[str, tuple[int, int]]]:
    if workflow_name == "A":
        return [{k: (MIN, MAX) for k in "xmas"}]
    elif workflow_name == "R":
        return []
    else:
        return ranges_for_workflow(workflows[workflow_name])


def apply_guard(g: Guard, rs: list[dict[str, tuple[int, int]]]) -> list[dict[str, tuple[int, int]]]:
    return [
        {
            var: (l, h) if var != g.var else (max(l, g.val + 1), h) if g.gt else (l, min(h, g.val - 1))
            for var, (l, h) in r.items()
        }
        for r in rs
    ]


def ranges_for_workflow(workflow: list[tuple[Guard | None, str]]) -> list[dict[str, tuple[int, int]]]:
    if len(workflow) == 1:
        return ranges_for_name(workflow[0][1])

    g, res = workflow[0]
    assert g is not None

    t = apply_guard(g, ranges_for_name(res))
    f = apply_guard(g.inverse(), ranges_for_workflow(workflow[1:]))
    return t + f

In [9]:
for r in ranges_for_name("in"):
    part_2 += math.prod(h - l + 1 for l, h in r.values())

In [10]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 434147

Part 2: 136146366355609